In [2]:
import pandas as pd# Load the DataFrame from the pickle file
pickle_file_path = 'C:/my_code/moduroad/data/obstacle/도보_턱.pkl'  # Replace with your pickle file path
obstacle_df = pd.read_pickle(pickle_file_path)
'''
# 모든 행의 데이터를 오른쪽으로 1칸씩 이동
obstacle_df = obstacle_df.shift(periods=1, axis='columns')

# 첫 번째 열 삭제 ('hg' 열이 이동 후 첫 번째 열임)
obstacle_df.drop(obstacle_df.columns[0], axis=1, inplace=True)

# 새로운 열 이름 지정 (예시에서 주어진 열 이름 사용)
new_columns = ['ctprvnnm', 'signgunm', 'signgucode', 'rdnmadr', 'lnmadr', 'latitude', 'longitude', 'institutionnm', 'phonenumeric', 'referencedate', 'instt_code', 'instt_nm', 'handicapcvntlyn', 'handicapcvntltype']
obstacle_df.columns = new_columns'''
#obstacle_df['signgucode'] = pd.to_numeric(obstacle_df['signgucode'], errors='coerce')
obstacle_df['latitude'] = pd.to_numeric(obstacle_df['latitude'], errors='coerce')
obstacle_df['longitude'] = pd.to_numeric(obstacle_df['longitude'], errors='coerce')
print(obstacle_df.dtypes)

obstacle_df

ctprvnnm            object
signgunm            object
signgucode          object
rdnmadr             object
lnmadr              object
latitude           float64
longitude          float64
institutionnm       object
referencedate       object
handicapcvntlyn     object
dtype: object


,ctprvnnm,signgunm,signgucode,rdnmadr,lnmadr,latitude,longitude,institutionnm,referencedate,handicapcvntlyn
0,인천광역시,중구,28110,NaN,인천광역시 중구 용유동 66-7,37.442268,126.376404,인천광역시,2021-12-03,N
1,인천광역시,중구,28110,NaN,인천광역시 중구 용유동 66-6,37.443530,126.376250,인천광역시,2021-12-03,N
2,인천광역시,계양구,28245,NaN,인천광역시 계양구 작전서운동 934,37.532056,126.735374,인천광역시,2021-12-03,N
3,인천광역시,계양구,28245,NaN,인천광역시 계양구 작전서운동 932,37.531759,126.734737,인천광역시,2021-12-03,N
4,인천광역시,서구,28260,NaN,인천광역시 서구 오류왕길동,37.568174,126.601612,인천광역시,2021-12-03,N
...,...,...,...,...,...,...,...,...,...,...
278,인천광역시,서구,28260,"인천광역시 서구 원적로 35-1, (가좌동)",인천광역시 서구 가좌3동 213,37.492649,126.680226,인천광역시,2021-12-03,N
279,인천광역시,부평구,28237,"인천광역시 부평구 부흥로 246, (부평동, 동아아파트)",인천광역시 부평구 부평1동 70-125,37.498710,126.718933,인천광역시,2021-12-03,N
280,인천광역시,부평구,28237,"인천광역시 부평구 경원대로 1376-2, (부평동, 가로판매대)",인천광역시 부평구 부평1동 126-10,37.497076,126.721589,인천광역시,2021-12-03,N
281,인천광역시,중구,28110,"인천광역시 중구 참외전로 131-15, (인현동)",인천광역시 중구 동인천동 1-617,37.475037,126.632117,인천광역시,2021-12-03,N


In [3]:
# obstacle_df DataFrame을 'C:/my_code/moduroad/data/obstacle/slope.pkl' 경로로 저장합니다.
obstacle_df.to_pickle('C:/my_code/moduroad/data/obstacle/도보_턱.pkl')

In [2]:
import pandas as pd
import osmnx as ox
from shapely.geometry import Point
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from shapely.strtree import STRtree
import pickle
import osmnx as ox
import folium
file_path = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph.pickle'

with open(file_path, 'rb') as file:
    G = pickle.load(file)
# Load the DataFrame from the pickle file
pickle_file_path = 'C:/my_code/moduroad/data/obstacle/도보_턱.pkl'  # Replace with your pickle file path
obstacle_df = pd.read_pickle(pickle_file_path)

G.graph['crs'] = 'epsg:4326'

# 장애물 가까이에 있는 엣지 가중치 증가 함수 정의
def update_edge_weights(G, point, attribute_name='sidewalk_curb', increment=1):
    # 가장 가까운 엣지 찾기
    
    nearest_edge_nodes = ox.distance.nearest_edges(G, X=point.x, Y=point.y)  # OSMnx는 (y, x) 형식으로 좌표를 사용합니다.
    
    # nearest_edge_nodes는 (u, v, key) 형태의 튜플입니다.
    u, v, key = nearest_edge_nodes
    
    # 해당 엣지의 데이터 가져오기
    nearest_edge = G[u][v][key]
    print(nearest_edge)
    # 가장 가까운 엣지의 가중치 업데이트
    # 시작 지점에 마커 추가
    
    if attribute_name in nearest_edge:
    # 속성 값이 0인지 확인합니다.
        if nearest_edge[attribute_name] == 0:
            # 속성 값이 0이라면, 새로운 포인트를 포함하는 리스트로 변경합니다.
            nearest_edge[attribute_name] = [point]
        elif isinstance(nearest_edge[attribute_name], list):
            # 속성 값이 리스트인 경우, 새로운 포인트를 리스트에 추가합니다.
            nearest_edge[attribute_name].append(point)
        else:
            # 속성 값이 0이 아니고 리스트도 아닌 다른 값인 경우, 그 값을 리스트에 넣고 새로운 포인트도 추가합니다.
            current_value = nearest_edge[attribute_name]
            nearest_edge[attribute_name] = [current_value, point]
            #print("change")
        print(nearest_edge)
    
        return (u, v, key), nearest_edge
    
i=0
# 모든 장애물에 대해 가중치 업데이트 함수 실행
for index, obstacle in obstacle_df.iterrows():
    latitude = obstacle['latitude']
    longitude = obstacle['longitude']
    print(i)
    # Point 객체 생성
    points= Point(longitude, latitude)
    # 가장 가까운 엣지의 가중치 업데이트 함수 적용
    updated_edge_info = update_edge_weights(G, points, increment=1)
    i+=1

# 저장할 파일 경로 지정
file_path_to_save = 'C:/my_code/moduroad/moduroad_app/cashe_data/api_graph_sidewalk_curb.pickle'

# api_graph 객체를 피클 파일로 저장
with open(file_path_to_save, 'wb') as file:
    pickle.dump(G, file)


0
{'link_id': '28100088738', 'geometry': <LINESTRING (126.376 37.442, 126.376 37.442, 126.376 37.442, 126.376 37.442,...>, 'length': 158.96164702896084, 'slope': 0, 'stair_steep': 0, 'sidewalk_curb': 0, 'crosswalk_curb': 0, 'bollard': 0, 'access_zone': 0, 'weight': 0}
{'link_id': '28100088738', 'geometry': <LINESTRING (126.376 37.442, 126.376 37.442, 126.376 37.442, 126.376 37.442,...>, 'length': 158.96164702896084, 'slope': 0, 'stair_steep': 0, 'sidewalk_curb': [<POINT (126.376 37.442)>], 'crosswalk_curb': 0, 'bollard': 0, 'access_zone': 0, 'weight': 0}
1
{'link_id': '28100088699', 'geometry': <LINESTRING (126.376 37.445, 126.376 37.445, 126.376 37.445, 126.376 37.444,...>, 'length': 122.0064729142875, 'slope': 0, 'stair_steep': 0, 'sidewalk_curb': 0, 'crosswalk_curb': 0, 'bollard': 0, 'access_zone': 0, 'weight': 0}
{'link_id': '28100088699', 'geometry': <LINESTRING (126.376 37.445, 126.376 37.445, 126.376 37.445, 126.376 37.444,...>, 'length': 122.0064729142875, 'slope': 0, 'stair_st